## Fine-tune textgenrnn weights on FDA drug name data

In [5]:
from textgenrnn import textgenrnn

In [67]:
textgen = textgenrnn.textgenrnn()
textgen.generate()

The police of the simple school song with a while of my same second days as the person in the middle of the most beautiful thing that won't be picking the table and in the piece of depression



In [68]:
textgen.train_from_file('fda-drug-names.txt', num_epochs=100)

3,662 texts collected.
Training on 31,181 character sequences.
Epoch 1/100
243/243 [==============================] - 23s 96ms/step - loss: 2.5050
####################
Temperature: 0.2
####################
PROMYCIN

PROPAR

PRESTIN

####################
Temperature: 0.5
####################
PREVASE

DEUREL

ELLOR

####################
Temperature: 1.0
####################
STOMSIPRO

TAROPE

ORFEC

Epoch 2/100
243/243 [==============================] - 23s 94ms/step - loss: 2.2743
####################
Temperature: 0.2
####################
ACTROLIN

SUPREX

SUPRAC

####################
Temperature: 0.5
####################
ACUVIR

REKEN

SUPRADER

####################
Temperature: 1.0
####################
BACMESTIZYME

STRAPGEL

TEFL

Epoch 3/100
243/243 [==============================] - 22s 90ms/step - loss: 2.1939
####################
Temperature: 0.2
####################
ACTICORT

CONOLIN

DESONE

####################
Temperature: 0.5
####################
AMBIVO

LOVIL

ATANYL

#####

## Save model weights

In [10]:
textgen.save(weights_path='textgenrnn_weights_10epoch.hdf5')

## Load model weights

In [1]:
from textgenrnn import textgenrnn

Using TensorFlow backend.


In [2]:
textgen = textgenrnn.textgenrnn(weights_path='textgenrnn_weights_10epoch.hdf5')
textgen.generate()

ARANEL



## Generate a lot and put them in a file

In [4]:
n_take = 1000
textgen.generate_to_file(f'textgenrnn_10epoch_{n_take}.txt', n=n_take,
                         temperature=0.75)

In [5]:
names = textgen.generate(n=1000, return_as_list=True, temperature=0.75)

In [6]:
names[:5]

['SOOLOXIN', 'PENCIN', 'PENTROL', 'FLUCINOLIN', 'KALECLOTE']

## Filter out words that are exactly in the training data

In [7]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
with open('fda-drug-names.txt', 'r') as f:
    data = f.readlines()
    data = [line.rstrip('\n') for line in data]

In [9]:
lookup = {}
for word in data:
    lookup[word] = True

In [10]:
unique_names = [name for name in names if name not in lookup]

In [11]:
len(unique_names)

970

## Find similar training data!

Get the most similar word in the training data.

In [12]:
def similarity(s1, s2):
    """Get similarity measure as negative Levenshtein distance.
    
    https://stackoverflow.com/questions/2460177/edit-distance-in-python
    """
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return -distances[-1]
    # return 1 / (float(distances[-1]) + 1e-5)

In [13]:
def iterate_over_train(word, train_set, k=3):
    similarities = []
    for train_word in train_set:
        similarities.append(similarity(word, train_word))
    
    # get indices to top K most similar words
    top_k_indices = np.argsort(similarities)[::-1][:k]
    
    # get the words
    top_k_words = [train_set[ix] for ix in top_k_indices]
    return top_k_words
    

In [14]:
name = np.random.choice(unique_names)
print(f'Searching for closest words to {name}')
print(iterate_over_train(name, data, k=10))

Searching for closest words to BROMPHERMOR
['BROMPHERIL', 'AEROSPHERE', 'COMPLERA', 'PROPHENE', 'BRONCHO', 'REMERON', 'PREMPRO', 'DISPERMOX', 'LOPRESSOR', 'COMPRO']
